## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。
RunnablePassthrough 类似一个分流器，可以让水流（数据）同时流向多个方向，而不会改变水的性质。
```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / |\
     /  |  \
 正面论述| 反面论述
     \  | /
      \ |/
     最终总结
```

In [5]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
# 初始化 ChatOpenAI 模型，指定使用的模型为 'gpt-4o-mini'
model = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_base="https://api.gptsapi.net/v1",
    openai_api_key="sk-XPp5a40638eec6f1ce1c0333e36bf6305e53c8ea95ccTUXc"
    )


# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | model 
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    |model
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    |model
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
    | model
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [ ]:
print(chain.invoke({"input": "房地产低迷"}))

#### 流式输出

In [11]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "全球经济"}):
    print(s, end="", flush=True)

在探索全球经济的正面和反面观点时，确实可以看到这两个方面如何相互交织，形成复杂而多层次的局面。以下是对此进行综合回应的总结:

1. **全球化的双刃剑**：
   虽然全球化促进了经济增长和国际合作，但也导致了某些地区和行业的衰退。为了平衡这一现象，各国需要采取积极的政策来帮助受到影响的行业和工人，推动再培训和技能提升，以帮助他们融入新的市场环境。

2. **不平等的复杂性**：
   全球化虽使一些国家受益匪浅，但也加剧了贫富差距。需要各国政府采取公正的税收政策和社会保障措施，帮助每个人分享经济增长的果实，以降低社会矛盾和不稳定风险。

3. **数字经济的挑战与机遇**：
   数字化带来了新的商业模式和成长机会，但同时也带来了隐私和就业的挑战。政策制定者应该在推动数字技术创新的同时，加强对数据保护的立法，确保公平竞争环境。

4. **应对气候变化的经济转型**：
   气候变化给经济带来了巨大挑战，但绿色转型也为创新和创造新就业机会提供了可能。通过政府、企业与社会的共同努力，投资可再生能源和可持续技术，将有助于构建一个更为稳健的经济体系。

5. **疫情后的复苏路径**：
   COVID-19疫情加剧了全球经济的不平等，富裕国家的复苏速度更快。国际社会需要加强合作与援助，尤其是对于发展中国家，从而实现更加均衡和可持续的全球经济复苏。

6. **地缘政治的复杂影响**：
   尽管地缘政治可能增加经济的不确定性，但也促使各国之间的对话与合作。各国应加强多边合作，寻找共同利益，以减少紧张关系所造成的经济风险。

7. **技术革命下的社会适应**：
   新技术的迅速发展可以提升生产力，但也可能导致劳动力结构的变化。社会需要提升对新技术的适应能力，加大教育与培训投资，帮助人们转向新型工作和领域。

最终，这些复杂的经济议题表明，全球经济的发展充满挑战，但仍蕴藏着巨大机遇。唯有在全面认识并应对负面影响的基础上，各国才能在全球化的浪潮中实现更包容、可持续的经济增长。同时，国际合作、政策创新和社会参与将是应对未来复杂经济挑战的关键因素。

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。